In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
#export
import pickle
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import random

In [ ]:
from sklearn.cluster import MiniBatchKMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import svm
from sklearn.mixture import GaussianMixture

In [ ]:
#export
train_images_filenames = pickle.load(open('Data/train_images_filenames.dat','rb'))
test_images_filenames = pickle.load(open('Data/test_images_filenames.dat','rb'))
train_labels = pickle.load(open('Data/train_labels.dat','rb'))
test_labels = pickle.load(open('Data/test_labels.dat','rb'))

Let us first read the train and test files

In [ ]:
#export
class Dataset():
    def __init__(self, images_filenames, labels):
        self.data = images_filenames
        self.labels = labels
        
    def visualize(self, samples_per_class=5):
        print(f'Total number of training samples: {len(self.data)}')
        # get unique classses
        classes = np.unique(np.array(self.labels))
        num_classes = len(classes)
        #set size for plot
        plt.figure(figsize=(15,8))

        for y, cls in enumerate(classes):
            _idxs = np.flatnonzero(np.array(self.labels) == cls)
            idxs = np.random.choice(_idxs, samples_per_class, replace=False)
            for i, idx in enumerate(idxs):
                plt_idx = i * num_classes + y + 1
                plt.subplot(samples_per_class, num_classes, plt_idx)
                plt.imshow(Image.open(self.data[idx]))
                plt.axis('off')
                if i == 0:
                    plt.title(f'{cls} ({len(_idxs)})')
        plt.show()
        
    def show_sample(self, indx):
        return Image.open(self.data[indx])

In [ ]:
train = Dataset(train_images_filenames, train_labels)
test = Dataset(test_images_filenames, test_labels)

We create a SIFT object detector and descriptor

In [ ]:
def load_pickle(path):
    """
    Retriev pickle file containing ground truths
    of query images
    """
    with open(path, 'rb') as f:
        query_gt = pickle.load(f)
    return query_gt

````python
with open('train_sift_descriptors.pkl', 'wb') as f:
    pickle.dump(Train_descriptors, f)
    
with open('test_sift_descriptors.pkl', 'wb') as f:
    pickle.dump(Test_descriptors, f)
````

## DENSE KEYPOINTS

In [ ]:
Train_descriptors_dense = load_pickle('train_sift_descriptors_dense.pkl')
Test_descriptors_dense = load_pickle('test_sift_descriptors_dense.pkl')

In [ ]:
Train_descriptors = load_pickle('train_sift_descriptors.pkl')
Test_descriptors = load_pickle('test_sift_descriptors.pkl')

## Fisher Vectors

Gaussian Mixture Models
size of k = 64
num_samples to take = 1000*k

In [ ]:
def fisher_vector(xx, gmm):
    """Computes the Fisher vector on a set of descriptors.
    Parameters
    ----------
    xx: array_like, shape (N, D) or (D, )
        The set of descriptors
    gmm: instance of sklearn mixture.GMM object
        Gauassian mixture model of the descriptors.
    Returns
    -------
    fv: array_like, shape (K + 2 * D * K, )
        Fisher vector (derivatives with respect to the mixing weights, means
        and variances) of the given descriptors.
    Reference
    ---------
    J. Krapac, J. Verbeek, F. Jurie.  Modeling Spatial Layout with Fisher
    Vectors for Image Categorization.  In ICCV, 2011.
    http://hal.inria.fr/docs/00/61/94/03/PDF/final.r1.pdf
    """
    xx = np.atleast_2d(xx)
    N = xx.shape[0]

    # Compute posterior probabilities.
    Q = gmm.predict_proba(xx)  # NxK

    # Compute the sufficient statistics of descriptors.
    Q_sum = np.sum(Q, 0)[:, np.newaxis] / N
    Q_xx = np.dot(Q.T, xx) / N
    Q_xx_2 = np.dot(Q.T, xx ** 2) / N

    # Compute derivatives with respect to mixing weights, means and variances.
    d_pi = Q_sum.squeeze() - gmm.weights_
    d_mu = Q_xx - Q_sum * gmm.means_
    d_sigma = (
        - Q_xx_2
        - Q_sum * gmm.means_ ** 2
        + Q_sum * gmm.covariances_
        + 2 * Q_xx * gmm.means_)

    # Merge derivatives into a vector.
    return np.hstack((d_pi, d_mu.flatten(), d_sigma.flatten()))

In [ ]:
from tqdm import tqdm

In [ ]:
def get_fisher_vectors(num_clusters_gmm, num_components_pca, Train_descriptors, Test_descriptors):
    """
    Returns train and test fisher vectors after training a GMM.
    
    """
    
    # sample a set of SIFT descriptors, precisely 1000*num_clusters_gmm
    D=np.vstack(Train_descriptors)
    indices = random.sample(range(0,D.shape[0]),num_clusters_gmm*1500)
    sample = D[indices,:]
    
    #apply PCA on them
    pca = PCA(n_components=num_components_pca)
    samplepca = pca.fit_transform(sample)
    
    # Fit a Gaussian mixture model on them, and compute mean, covariance and weight matrix
    gmm = GaussianMixture(n_components=num_clusters_gmm,covariance_type='diag')
    gmm.fit(samplepca)
    
    Train_descriptors_fisher = []
    for train_descriptor in tqdm(Train_descriptors):
        start = 0
        length = train_descriptor.shape[0]
        stop = start + length
        train_descriptor_pca = train_descriptor[:,:num_components_pca]
        start = stop
        train_descriptor_fisher = fisher_vector(train_descriptor_pca, gmm)
        Train_descriptors_fisher.append(train_descriptor_fisher)
        
    Dtest = np.vstack(Test_descriptors)
    Test_descriptors_fisher = []
    for test_descriptor in tqdm(Test_descriptors):
        start = 0
        length = test_descriptor.shape[0]
        stop = start + length
        test_descriptor_pca = test_descriptor[:,:num_components_pca]
        start = stop
        test_descriptor_fisher = fisher_vector(test_descriptor_pca, gmm)
        Test_descriptors_fisher.append(test_descriptor_fisher)
    
    def normalize_fisher(descriptors):
        image_fvs = np.vstack(descriptors)
        image_fvs = np.sign(image_fvs) * np.abs(image_fvs) ** 0.5
        norms = np.sqrt(np.sum(image_fvs ** 2, 1))
        image_fvs /= norms.reshape(-1, 1)
        return image_fvs
    
    train_fisher = normalize_fisher(Train_descriptors_fisher)
    test_fisher = normalize_fisher(Test_descriptors_fisher)
    return train_fisher, test_fisher

In [ ]:
train_f, test_f = get_fisher_vectors(num_clusters_gmm=16, num_components_pca=32, 
                                    Train_descriptors_dense, Test_descriptors_dense)

100%|██████████| 807/807 [00:00<00:00, 844.91it/s] 


### Now Build A Classifier

In [ ]:
knn = KNeighborsClassifier(n_neighbors=10,n_jobs=-1,metric='euclidean')
knn.fit(train_f, train_labels) 
accuracy = 100*knn.score(test_f, test_labels)
print(accuracy)

74.8451053283767


In [ ]:
classifier = svm.SVC()
classifier.fit(train_f, train.labels)
accuracy = 100*classifier.score(test_f, test.labels)
print(accuracy)

79.92565055762083
